In [ ]:
import model_server

In [2]:
OnlineData = model_server.DATA(ReadALLSubjs=True,SplitData=True)

In [3]:
OnlineData.scandata()

Scaning the AB193 subject is done.
Scaning the AB186 subject is done.
Scaning the AB192 subject is done.
Scaning the AB156 subject is done.
Scaning the AB188 subject is done.
Scaning the AB190 subject is done.
Scaning the AB194 subject is done.
Scaning the AB185 subject is done.
Scaning the AB191 subject is done.
Scaning the AB189 subject is done.


In [4]:
OnlineData.stackdata(ChooseSensors=['IMU','GONIO'])

38 channels of ['IMU', 'GONIO'] data are selected:
Start stacking data...(Split data into train set and val set), and split ratio is  0.33
Finish stacking data of the 1 subjects: 50 trails
Finish stacking data of the 2 subjects: 44 trails
Finish stacking data of the 3 subjects: 48 trails
Finish stacking data of the 4 subjects: 50 trails
Finish stacking data of the 5 subjects: 40 trails
Finish stacking data of the 6 subjects: 49 trails
Finish stacking data of the 7 subjects: 50 trails
Finish stacking data of the 8 subjects: 52 trails
Finish stacking data of the 9 subjects: 44 trails
Finish stacking data of the 10 subjects: 49 trails
Done stacking data! 
Train dataset: (8013012, 38) 
Val dataset: (3705714, 38)


In [23]:
OnlineData.PreprocessData('AbsMax')

The preprocess method is AbsMax


MemoryError: 

In [5]:
OnlineData.displayConfig()


Configuration:
CHOOSEDCHANNELS                [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 44, 45, 46, 47, 48, 49, 50, 51]
DATAPATH                       ../../../../Dataset
DataType                       ['Features', 'MVC', 'Processed', 'Raw']
INDEX_EMG                      [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]
INDEX_GONIO                    [44, 45, 46, 47, 48, 49, 50, 51]
INDEX_IMU                      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
IfReadAll                      True
IfSplitData                    True
LABELCATEGORIES                ['Sitting', 'Level Ground Walking', 'Ramp Ascent', 'Ramp Descent', 'Stair Ascent', 'Stair Descent', 'Standing']
LEARNING_RATE                  0.001
MAXABS_PREPROCESS              False
SCANEDSubjects                 None
SPILTRATIO                     0.33
STEPS_PER_EPOCH                10

In [7]:
OnlineData.TRAIN_DATA_all.shape

(8013012, 38)

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model_rnn = Sequential()
model_rnn.add(LSTM(512, input_shape=(None, OnlineData.TRAIN_DATA_all.shape[1]),
                    dropout=0.5))
model_rnn.add(Dense(OnlineData.TRAIN_LABEL_all.shape[1], activation='softmax'))
model_rnn.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 512)               1128448   
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 3591      
Total params: 1,132,039
Trainable params: 1,132,039
Non-trainable params: 0
_________________________________________________________________


In [ ]:
TenEpochesWithDropout = model_rnn.fit(
    OnlineData.TRAIN_DATA_all.reshape((OnlineData.TRAIN_DATA_all.shape[0], 1, OnlineData.TRAIN_DATA_all.shape[1])),
    OnlineData.TRAIN_LABEL_all, epochs=10,
    validation_data=(
        OnlineData.VAL_DATA_all.reshape((OnlineData.VAL_DATA_all.shape[0], 1, OnlineData.VAL_DATA_all.shape[1])),
        OnlineData.VAL_LABEL_all))